<a href="https://colab.research.google.com/github/tamayodb/ndrrmc-typhoon-data-preprocess/blob/main/ndrrmc_typhoon_data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()


Saving 2020-QUINTA.xlsx to 2020-QUINTA (1).xlsx


In [5]:
import pandas as pd
import numpy as np
import re
from difflib import SequenceMatcher
from datetime import datetime

def normalize_location_name(name, keep_parentheses=False):
    """Comprehensive location name normalization with parentheses handling"""
    if pd.isna(name):
        return name

    name = str(name).strip()

    # Remove common prefixes/suffixes
    name = re.sub(r'^(city of|municipality of|province of)\s+', '', name, flags=re.IGNORECASE)

    # Handle parentheses - extract base name unless keep_parentheses=True
    if not keep_parentheses:

        base_name = re.sub(r'\s*\([^)]*\).*', '', name).strip()
        if base_name:
            name = base_name

    # Standardize separators
    name = name.replace('-', ' ')
    name = name.replace('_', ' ')
    name = name.replace('.', '')

    # Handle common abbreviations
    abbreviations = {
        ' st ': ' saint ',
        ' st.': ' saint',
        ' sto ': ' santo ',
        ' sto.': ' santo',
        ' sta ': ' santa ',
        ' sta.': ' santa',
        ' n ': ' north ',
        ' s ': ' south ',
        ' e ': ' east ',
        ' w ': ' west ',
    }

    name_lower = name.lower()
    for abbr, full in abbreviations.items():
        name_lower = name_lower.replace(abbr, full)

    # Remove extra whitespace and standardize case
    name = ' '.join(name_lower.split())
    name = name.title()  # Proper case

    return name

def create_location_mapping(df):
    """Create a mapping for similar location names to handle parentheses cases"""
    location_col = 'City/Municipality'
    if location_col not in df.columns:
        return {}

    locations = df[location_col].dropna().unique()
    mapping = {}

    for loc in locations:
        base_name = normalize_location_name(loc, keep_parentheses=False)
        if base_name != loc:
            mapping[loc] = base_name
            print(f" Mapping: '{loc}' → '{base_name}'")

    return mapping

def clean_typhoon_name(name):
    """Clean and standardize typhoon names"""
    if pd.isna(name):
        return name

    name = str(name).strip().upper()

    # Remove common prefixes
    name = re.sub(r'^(TYPHOON|TY|TROPICAL STORM|TS)\s+', '', name)

    # Handle parentheses and additional info
    name = re.sub(r'\s*\([^)]*\)', '', name)
    name = re.sub(r'\s*\d{4}.*', '', name)

    return name.strip()

def standardize_numeric_columns(df, numeric_cols):
    """Clean and standardize numeric columns"""
    for col in numeric_cols:
        if col not in df.columns:
            continue

        # Convert to string first to handle mixed types
        df[col] = df[col].astype(str)

        # Remove common non-numeric characters
        df[col] = df[col].str.replace(',', '')  # Remove commas
        df[col] = df[col].str.replace('₱', '')  # Remove peso sign
        df[col] = df[col].str.replace('PHP', '', case=False)
        df[col] = df[col].str.replace('$', '')
        df[col] = df[col].str.replace(' ', '')  # Remove spaces

        # Handle common text values
        df[col] = df[col].str.replace('none', '0', case=False)
        df[col] = df[col].str.replace('nil', '0', case=False)
        df[col] = df[col].str.replace('n/a', '0', case=False)
        df[col] = df[col].str.replace('na', '0', case=False)
        df[col] = df[col].str.replace('-', '0')

        # Convert to numeric, replacing non-convertible with 0
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

        # Handle negative values (set to 0 as they're likely data entry errors)
        df[col] = df[col].clip(lower=0)

    return df

def clean_assistance_type(assistance_type):
    """Standardize assistance type names"""
    if pd.isna(assistance_type):
        return assistance_type

    assistance_type = str(assistance_type).strip()

    # Standardize common assistance types
    type_mapping = {
        'ffp': 'Family Food Pack',
        'family food packs': 'Family Food Pack',
        'food pack': 'Family Food Pack',
        'relief goods': 'Relief Goods',
        'relief good': 'Relief Goods',
        'financial assistance': 'Financial',
        'cash assistance': 'Financial',
        'financial aid': 'Financial',
        'medicine': 'Medical Assistance',
        'medical supplies': 'Medical Assistance',
        'hygiene kit': 'Hygiene Kit',
        'hygiene kits': 'Hygiene Kit',
        'sleeping kit': 'Sleeping Kit',
        'sleeping kits': 'Sleeping Kit',
    }

    assistance_lower = assistance_type.lower()
    for key, value in type_mapping.items():
        if key in assistance_lower:
            return value

    # If no mapping found, return title case
    return assistance_type.title()

def validate_and_clean_year(year):
    """Validate and clean year values"""
    if pd.isna(year):
        return year

    try:
        year = int(float(str(year)))
        # Reasonable range for typhoon data
        if 2020 <= year <= datetime.now().year:
            return year
        else:
            return np.nan
    except:
        return np.nan

def preprocess_dataframe(df, sheet_name):
    """Apply comprehensive preprocessing to a dataframe"""
    print(f"  Preprocessing {sheet_name}...")

    # Store original shape
    original_shape = df.shape

    # 1. Clean column names
    df.columns = df.columns.str.strip()

    # 2. Remove completely empty rows and columns
    df = df.dropna(how='all')  # Remove empty rows
    df = df.loc[:, df.notna().any()]  # Remove empty columns

    # 3. Create location mapping for parentheses cases
    if 'City/Municipality' in df.columns:
        location_mapping = create_location_mapping(df)
        # Apply the mapping
        df['City/Municipality'] = df['City/Municipality'].replace(location_mapping)

    # 4. Standardize location names
    location_cols = ['Province', 'City/Municipality', 'Region']
    for col in location_cols:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: normalize_location_name(x, keep_parentheses=False))

    # 5. Clean typhoon names
    if 'Typhoon Name' in df.columns:
        df['Typhoon Name'] = df['Typhoon Name'].apply(clean_typhoon_name)

    # 6. Validate and clean years
    if 'Year' in df.columns:
        df['Year'] = df['Year'].apply(validate_and_clean_year)

    # 7. Handle numeric columns based on sheet type
    numeric_cols = []
    if sheet_name == "Affected Population":
        numeric_cols = ['Families', 'Person', 'Brgy']
    elif sheet_name == "Casualties":
        numeric_cols = ['Dead', 'Injured/Ill', 'Missing']
    elif sheet_name == "Damaged Houses":
        numeric_cols = ['Totally', 'Partially', 'Total']
    elif sheet_name == "Assistance Provided":
        numeric_cols = ['Quantity', 'Cost']
        # Special handling for assistance type
        if 'Type' in df.columns:
            df['Type'] = df['Type'].apply(clean_assistance_type)

    df = standardize_numeric_columns(df, numeric_cols)

    # 8. Remove duplicate records (but aggregate them properly)
    key_cols = ['Typhoon Name', 'Year', 'Region', 'Province', 'City/Municipality']
    available_keys = [col for col in key_cols if col in df.columns]

    if sheet_name != "Assistance Provided":  # Don't remove duplicates from assistance data
        # Instead of just dropping duplicates, aggregate them
        if available_keys:
            numeric_cols_in_df = [col for col in numeric_cols if col in df.columns]
            text_cols_in_df = [col for col in df.columns if col not in available_keys + numeric_cols_in_df]

            agg_funcs = {}
            for col in numeric_cols_in_df:
                agg_funcs[col] = 'sum'
            for col in text_cols_in_df:
                agg_funcs[col] = lambda x: ', '.join(x.dropna().astype(str).unique()) if len(x.dropna()) > 0 else np.nan

            if agg_funcs:
                before_agg = len(df)
                df = df.groupby(available_keys, as_index=False).agg(agg_funcs)
                after_agg = len(df)
                if before_agg != after_agg:
                    print(f" Aggregated {before_agg} records into {after_agg} records")

    # 9. Data validation
    validation_issues = []

    # Check for missing key information
    for col in available_keys:
        missing_count = df[col].isna().sum()
        if missing_count > 0:
            validation_issues.append(f"{col}: {missing_count} missing values")

    # Check for outliers in numeric columns
    for col in numeric_cols:
        if col in df.columns:
            q99 = df[col].quantile(0.99)
            if q99 > 0:  # Avoid division by zero
                outliers = (df[col] > q99 * 10).sum()  # Values 10x larger than 99th percentile
                if outliers > 0:
                    validation_issues.append(f"{col}: {outliers} potential outliers")

    if validation_issues:
        print(f"    Data quality issues: {'; '.join(validation_issues)}")

    print(f"   Shape: {original_shape} → {df.shape}")

    return df

# Main processing
file_path = "/content/2020-QUINTA.xlsx"
xls = pd.ExcelFile(file_path)

print("Starting comprehensive data preprocessing...")

# --- Define sheets we want ---
sheets = {
    "Affected Population": ["Families", "Person", "Brgy"],
    "Casualties": ["Dead", "Injured/Ill", "Missing"],
    "Damaged Houses": ["Totally", "Partially", "Total"],
}

# --- Key columns for merging ---
key_cols = ["Typhoon Name", "Year", "Region", "Province", "City/Municipality"]

merged = None

# Process sheets that can be aggregated
for sheet, cols in sheets.items():
    print(f"\n Processing {sheet}...")
    df = pd.read_excel(file_path, sheet_name=sheet)

    # Apply preprocessing
    df = preprocess_dataframe(df, sheet)

    # Keep only relevant columns
    keep_cols = [c for c in key_cols + cols if c in df.columns]
    df = df[keep_cols]

    # Group by keys for final aggregation
    numeric_cols = [c for c in cols if c in df.columns and df[c].dtype in ['int64', 'float64']]
    text_cols = [c for c in cols if c in df.columns and df[c].dtype == 'object']

    agg_funcs = {}
    for col in numeric_cols:
        agg_funcs[col] = 'sum'
    for col in text_cols:
        agg_funcs[col] = lambda x: ', '.join(x.dropna().astype(str).unique())

    if agg_funcs:
        df = df.groupby([col for col in key_cols if col in df.columns], as_index=False).agg(agg_funcs)
    else:
        df = df.drop_duplicates(subset=[col for col in key_cols if col in df.columns])

    # Merge with main table
    if merged is None:
        merged = df
    else:
        merge_keys = [col for col in key_cols if col in merged.columns and col in df.columns]
        merged = pd.merge(merged, df, on=merge_keys, how="outer")

# Process assistance data
print(f"\n Processing Assistance Provided...")
assistance_df = pd.read_excel(file_path, sheet_name="Assistance Provided")
assistance_data = preprocess_dataframe(assistance_df, "Assistance Provided")

# Keep relevant assistance columns
assistance_cols = ["Quantity", "Type", "Cost"]
assistance_keep_cols = [c for c in key_cols + assistance_cols if c in assistance_data.columns]
assistance_data = assistance_data[assistance_keep_cols]

# Final merge
print(f"\n Performing final merge...")
merge_keys = [col for col in key_cols if col in merged.columns and col in assistance_data.columns]
final_merged = pd.merge(merged, assistance_data, on=merge_keys, how="outer")

# Final data quality report
print(f"\n Final Data Quality Report:")
print(f"  • Total records: {len(final_merged):,}")
print(f"  • Unique typhoons: {final_merged['Typhoon Name'].nunique()}")
print(f"  • Year range: {final_merged['Year'].min():.0f} - {final_merged['Year'].max():.0f}")
print(f"  • Provinces covered: {final_merged['Province'].nunique()}")
print(f"  • Cities/Municipalities: {final_merged['City/Municipality'].nunique()}")

if 'Type' in final_merged.columns:
    print(f"  • Assistance types: {final_merged['Type'].nunique()}")
    print(f"  • Top assistance types: {final_merged['Type'].value_counts().head(3).to_dict()}")

# Save results | Naming convention - merged_typhoon_year_data_cleaned
final_merged.to_excel("/content/merged_typhoon_2020_data_cleaned.xlsx", index=False)
print("Cleaned and merged file saved: merged_typhoon_2020_data_cleaned.xlsx")

# Show sample for verification
print(f"\n Sample of cleaned data:")
sample_cols = ['Typhoon Name', 'Year', 'Province', 'City/Municipality', 'Type', 'Quantity', 'Cost']
display_cols = [col for col in sample_cols if col in final_merged.columns]
print(final_merged[display_cols].head(5))

Starting comprehensive data preprocessing...

 Processing Affected Population...
  Preprocessing Affected Population...
 Mapping: 'Dingalan ' → 'Dingalan'
 Mapping: 'Mendez (Mendez-Nuñez)' → 'Mendez'
 Mapping: 'Tobias Fornier (Dao)' → 'Tobias Fornier'
 Mapping: 'Sta. Cruz' → 'Sta Cruz'
 Mapping: 'Sta. Fe' → 'Sta Fe'
 Mapping: 'Sta. Maria' → 'Sta Maria'
 Mapping: 'Rapu-Rapu' → 'Rapu Rapu'
 Mapping: 'Pio V. Corpuz' → 'Pio V Corpuz'
 Mapping: 'Sta. Magdalena' → 'Sta Magdalena'
 Mapping: 'Laua-an' → 'Laua An'
 Mapping: 'San Jose de Buenavista' → 'San Jose De Buenavista'
 Mapping: 'Hinoba-an (Asia)' → 'Hinoba An'
 Mapping: 'Moises Padilla (Magallon)' → 'Moises Padilla'
 Mapping: 'Bayawan (Tulong)' → 'Bayawan'
   Shape: (337, 8) → (337, 8)

 Processing Casualties...
  Preprocessing Casualties...
 Mapping: 'Sanchez-Mira' → 'Sanchez Mira'
 Mapping: 'San Andres (Calolbon)' → 'San Andres'
 Mapping: 'Tobias Fornier (Dao)' → 'Tobias Fornier'
 Aggregated 56 records into 35 records
   Shape: (141, 8